# Cash-Karp Runge-Kutta Method with adaptive stepwise control

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Defining the coupled ODEs

In [ ]:
def dydx(x,y):
    
    #set the derivatives
    
    #our eqn is d^2y/dx^2 = -y
    #therefore represent 2nd order as 2 first order ODEs
    #dy/dx = z
    #dz/dx = -y
    
    #set y=y[0]  <-this is the 1st array element
    #set z=y[1]  <- 2nd array element
    
    #declare an array
    y_derivs = np.zeros(2) #a 1x2 array of value 0 -> [y=0,z=0]
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]  #-y
    
    #here we have to return an array
    #returns an array with dy/dx and dz/dx
    return y_derivs

## Define the Cash-Karp Runge-Kutta scheme used to evolve multiple variables at a time

In [ ]:
#scheme for multiple variables - need arrays
def ckrk_mv_core(dydx,xi,yi,nv,h):  #mv for multiple variables
    
    #dydx is the function of derivatives
    #xi is the current value of x at step i
    #yi is the array of variables at step i
    #nv is the number of variables
    #h is the step size
    
    #Creating arrays for magical adaptive step-size coefficients
    #making arrays larger than necessary to easy equation index matching
    #matrix a will be MxN
    Mi = 7 #this will let arrays have elements 0 to 6
    Nj = 6 #element 0 to 5
    
    #matrix A will be MxN
    A_MN = np.zeros((Mi,Nj))
    #matrix B, B_star and C will be Mx1
    B_M = np.zeros(Mi)
    B_star_M = np.zeros(Mi)
    C_M = np.zeros(Mi)
    
    #Setting A, B, B_star, C matrices equal to constants
    A_MN[2,1] = 1./5.
    A_MN[3,1] = 3./40.
    A_MN[3,2] = 9./40.
    A_MN[4,1] = 3./10.
    A_MN[4,2] = -9./10.
    A_MN[4,3] = 6./5.
    A_MN[5,1] = -11./54.
    A_MN[5,2] = 5./2.
    A_MN[5,3] = -70./27.
    A_MN[5,4] = 35./27.
    A_MN[6,1] = 1631./55296.
    A_MN[6,2] = 175./512.
    A_MN[6,3] = 575./13824.
    A_MN[6,4] = 44275./110592.
    A_MN[6,5] = 253./4096.
    
    
    
    #define x at 1/2 step - halfway point
    x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    x_ipo = xi + h
    
    #declare a temp y array to contain estimates of yi in different 
    #places within the interval
    y_temp = np.zeros(nv)
    
    #get k1 values
    #computing dydx on the left side of the interval to start
    y_derivs = dydx(xi,yi)  #values of the derivatives of x and y at step i
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    #first midpoint guess
    y_temp[:] = yi[:] + 0.5*k1[:]  #estimate of y halfway across the interval
    #now use new approz to calc new deriv
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    #another midpoint approx, now using estimates at the midpoint 
    #to calc another midpoint approx (dependent on k2)
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    #estimate of the deriv on the right side of the interval
    #extrapolating from our midpoint approx for deriv
    y_temp[:] = yi[:] + k3[:] #taking a Full step + k3
    y_derivs = dydx(x_ipo,y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance all values of y by a step h
    #can do y and z simultaneously at every step
    #weight all estimates together
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

## Define an adaptive step size driver for CKRK

In [ ]:
#can use information about the function and an error estimate of
#the function such that you can take different size steps
#depending on the shape of the function
#and still retain the needed level of accurary

def ckrk_mv_ad(dydx,x_i,y_i,nv,h,tol):
    #nv = number of variables
    # h = current step
    #tol = tolerance
    
    #define safety scale in order to appropriately change step
    #if current error estimate is small, can take bigger step
    #and still reach same level of accurary.
    #if we check our error estimate, we need to redo our current step
    #with a smaller step size to maintain level of accuracy
    
    #we know what overall tolerance is, this helps us decide how big of
    #a step to take in a complicated system of coupled ODEs (could be worse)
    
    #define safety scale, avoids raising an error in the first place
    #comes from Numerical Recipes
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error array to tell us current error estimate
    #for each variable
    #Recall integrating 2 variables at a time
    Delta = np.full(nv,2*tol)  #nv=2
    
    #starting with error bigger than tol so that while loop executes
    #and adjusts our stepsize
    
    #remember the step
    #check the first guess
    h_step = h
    
    #adjust step if necessary (to take smaller steps)
    #maintains our high level of accurary based on inputted tolerance
    while(Delta.max()/tol > 1.0): #takes element with max value in Delta and checks if larger than tol
        
       "Define for ckrk how to get error"
    
        #if the error is too large, take a smaller step
        if (Delta.max()/tol > 1.0):
            #our error is too large, decrease and retake the step
            #can never exceed tolerance - DANGER - diverges
            #if exceed, throw step away and retake
            #adjusts step size until within tolerance
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25) #neg power of # greater than 1 ->smaller
            
        #check iteration
        if(i>=imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i=",i)
        
        #iterate
        i+=1
    
    #if out of loop, have successfully taken step that is w/in tolerance    
    #next time, try to take a bigger step to improve efficiency
    #fmin gives next possible step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

## Define a wrapper CKRK

In [ ]:
def ckrk_mv(dydx,a,b,y_a,tol):
    #integrates over the interval, each step adaptively size to maintain
    #the order of error
    
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a is the boundary condition at a
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy() #values of y over a
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled odes to the
    #size of y_a
    nv = len(y_a)  #2
    
    #set the initial conditions
    x = np.full(1,a)  #array with single value, a
    y = np.full((1,nv),y_a) #2d array, 1x2, initialized to y_a
    #since dont initially know where taking values of y, going to
    #append new elements to array as steps are adaptively taken
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1 using an adaptive step
        yi_new, h_new, h_step = ckrk_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #imagine close to b, need to prevent an overshoot
        if(xi+h_step>b):
       
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = ckrk_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break, have reached the right side
            flag = 0
            
        #if did not enter if statement
        #update the values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        #cannot simply append to a 2d array
        y_new = np.zeros((len(x),nv)) #making new array the length of x
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]  #setting last elements to new y approximations
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number =", i)
            
        #iterate 
        i += 1
        
        #output some information
        #3d means 3 digits
        #9.8f means float with 9 digits, 8 after the decimal point
        # \t tab
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb = %9.8f" %(i,xi,h_step,b)
        print(s)
        
        #break if new xi is == b, the upper bound
        if(xi==b):
            flag = 0
    
    #out of the while loop
    #return the answer
    return x,y

## Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0  #sin(0)=0
y_0[1] = 1.0   #initial derivative is cosine, cos(0)=1
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = ckrk_mv(dydx,a,b,y_0,tolerance)

## Plot the results

In [ ]:
# graphing shows when steps need to be taking adaptively
#can typically taken large steps when change in derivative small
#small steps with derivitative changes rapidly
#however, these are coupled, sin is curves while cosine is relatively straight
#each step needs to be adjusted accordingly
#where can allow larger step in sine, need smaller step for cosine
#Amazing: have a tabulated set of values of the function everywhere along the way
#along the whole domain

plt.plot(x,y[:,0],'o',label = 'y(x)') #plotting all element of first set of y
plt.plot(x,y[:,1],'o',label = 'dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)  #array of 0 to 2pi in 1000 equal steps
plt.plot(xx,np.sin(xx),label="sin(x)")
plt.plot(xx,np.cos(xx),label="cos(x)")
plt.xlabel('x')
plt.ylabel('y,dy/dx')
plt.legend(frameon=False)

## Plot the absolute error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x) #eval sin and cos at the locations we computed solns to ODEs

y_error = (y[:,0]-sine) #y array - sine
dydx_error = (y[:,1]-cosine) #z array - cosine

plt.plot(x,y_error, label="y(x) Error")
plt.plot(x,dydx_error, label="dydx(x) Error")
plt.legend(frameon = False)

#error is oscillatory, larger than smaller